In [1]:
%pip install -qU llama-index datasets pinecone-client openai transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.6/604.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
from datasets import load_dataset

# Loading in the Squad dataset which contains 18891 rows
df = load_dataset('squad', split='train')

# We will only be using the id, context, and title columns to work with our dataset
df = df.to_pandas()[['id', 'context', 'title']]

df.drop_duplicates(subset='context', keep='first', inplace=True)

# Taking a random sample of 10000 rows to shorten the dataset we're working with
df = df.sample(n=10000, random_state=12)

In [33]:
df.head()

,id,context,title
62588,5728062d3acd2414000df281,"In mathematics, a group is an algebraic struct...",Group_(mathematics)
64911,572996a1af94a219006aa543,Many adult insects use six legs for walking an...,Insect
59636,5727beae2ca10214002d9546,As in other major American cities in the postw...,Detroit
38577,5725d04f89a1e219009abf30,Greeks in pre-Roman Gaul were mostly limited t...,Hellenistic_period
66420,5728fc671d04691400778f07,In the 1500s a new type of armor started to be...,Samurai


In [9]:
from llama_index import Document

documents = []

# Populating our list of documents with the text, doc_id, and extra_info with the 3 columns in our dataframe
for index, row in df.iterrows():
    documents.append(Document(
        text=row['context'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))

documents[0]

Document(id_='572ea842dfa6aa1500f8d26f', embedding=None, metadata={'title': 'Neptune'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='59c933b0f27dacde46b1d5db9d9e8531f21a0eddbeeeafdf22c86bf77d2d6804', text="Neptune's more varied weather when compared to Uranus is due in part to its higher internal heating. Although Neptune lies over 50% further from the Sun than Uranus, and receives only 40% its amount of sunlight, the two planets' surface temperatures are roughly equal. The upper regions of Neptune's troposphere reach a low temperature of 51.8 K (−221.3 °C). At a depth where the atmospheric pressure equals 1 bar (100 kPa), the temperature is 72.00 K (−201.15 °C). Deeper inside the layers of gas, the temperature rises steadily. As with Uranus, the source of this heating is unknown, but the discrepancy is larger: Uranus only radiates 1.1 times as much energy as it receives from the Sun; whereas Neptune radiates about 2.61 times as much energy as

In [13]:
from llama_index.node_parser import SimpleNodeParser

# Splitting up our documents into nodes, each containing a different chunk of text
parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)
nodes[0]

TextNode(id_='bb3a90a6-9d02-40ba-a7eb-e6a84e2a32e3', embedding=None, metadata={'title': 'Affirmative_action_in_the_United_States'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='57273936dd62a815002e99b4', node_type=None, metadata={'title': 'Affirmative_action_in_the_United_States'}, hash='42996f2c6b6d08c60088834dd10af3c08ba272e4d3b6f478c0b43eb1cc622029')}, hash='42996f2c6b6d08c60088834dd10af3c08ba272e4d3b6f478c0b43eb1cc622029', text='After the Nixon administration, advancements in affirmative action became less prevalent. "During the brief Ford administration, affirmative action took a back seat, while enforcement stumbled along.":145 Equal rights was still an important subject to many Americans, yet the world was changing and new issues were being raised. People began to look at affirmative action as a glorified issue of the past and now there were other areas that needed focus. "Of all the trium

In [15]:
import pinecone

# Setting our Pinecone API key and ENV to interact with our index
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

In [16]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

# Setting the index name
index_name = ''

# connect to the index
pinecone_index = pinecone.Index(index_name)

In [21]:
from llama_index.vector_stores import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [ ]:
import openai

# Setting the OpenAI API Key to create the embeddings for our documents
openai.api_key = ''

In [26]:
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# Setting up our vector store (Pinecone)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

# Setup the index process which we will use to query our documents
embedding_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)

service_context = ServiceContext.from_defaults(embed_model=embedding_model)

index = GPTVectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    service_context=service_context
)

Upserted vectors:   0%|          | 0/10000 [00:00<?, ?it/s]

In [30]:
# Creating a query engine using our index created from the GPTVectorStoreIndex
query_engine = index.as_query_engine()
res = query_engine.query("Describe the weather on Neptune")
print(res)


The weather on Neptune is highly active and visible, with strong sustained winds reaching speeds of up to 2,100 kilometres per hour (580 m/s; 1,300 mph). At the time of the Voyager 2 flyby in 1989, the planet's southern hemisphere had a Great Dark Spot comparable to the Great Red Spot on Jupiter. The upper regions of Neptune's troposphere reach a low temperature of 51.8 K (−221.3 °C). At a depth where the atmospheric pressure equals 1 bar (100 kPa), the temperature is 72.00 K (−201.15 °C). Deeper inside the layers of gas, the temperature rises steadily.
